## Módulo 1

In [1]:
#Importando as bibliotecas

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

iris = load_iris()
x = iris.data #[n_amostras, n_features]
y = iris.target # Vetor de alvo [n_amostras]

print("Forma das features (x): ", x.shape)
print("Forma das caracteristicas (x): ", x.shape[1])
print("Forma do valor alvo (y): ", y.shape)

Forma das features (x):  (150, 4)
Forma das caracteristicas (x):  4
Forma do valor alvo (y):  (150,)


### Treino e Teste